El modelo se encuentra en el repositorio

In [1]:
!git clone https://github.com/UN-GCPDS/eeg_models.git

Cloning into 'eeg_models'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 216 (delta 0), reused 0 (delta 0), pack-reused 213 (from 1)
Receiving objects: 100% (216/216), 381.66 KiB | 7.95 MiB/s, done.
Resolving deltas: 100% (142/142), done.


Creamos la señal arbitraria de EEG

In [2]:
import numpy as np

n_channels = 32
duration_sec = 10
sampling_rate = 250
n_samples = duration_sec * sampling_rate

X = np.random.randn(1000, n_channels, n_samples) # señal de EEG
y = np.eye(2)[np.random.randint(0, 2, size=1000)]

Cargamos el modelo y sus respectivos argumentos

In [3]:
import sys
sys.path.append('/kaggle/working/eeg_models')
from models import ShallowConvNet

# Parámetros específicos del modelo
model_params = {
    'nb_classes': 2,
    'dropoutRate': 0.5,
    'version': '2018'
}

# Argumentos generales del modelo
model_args = {
    'model_name': 'ShallowConvNet',
    'nb_classes': 2,
    'autoencoder': False
}

# Inicialización del modelo con los parámetros definidos
model = ShallowConvNet(**model_params, Chans=n_channels, Samples=n_samples)

model.summary()

2025-07-22 14:38:39.821704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753195120.037598      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753195120.101236      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-22 14:38:56.325431: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 2500, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2D_1 (Conv2D)               │ (None, 32, 2488, 40)   │           560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2D_2 (Conv2D)               │ (None, 1, 2488, 40)    │        51,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1, 2488, 40)    │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1, 2488, 40)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 1, 351, 40)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 1, 351, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 351, 40)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14040)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 2)              │        28,082 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out_activation (Activation)     │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,042 (312.66 KB)

 Trainable params: 79,962 (312.35 KB)

 Non-trainable params: 80 (320.00 B)

Compilamos el modelo y lo entrenamos con los datos

In [4]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

compile_args = {
    'loss': BinaryCrossentropy(),
    'metrics': ['binary_accuracy']
}
model.compile(**compile_args)

model.fit(X,y, epochs=10)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - binary_accuracy: 0.5387 - loss: 0.6905
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - binary_accuracy: 0.9065 - loss: 0.5281
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - binary_accuracy: 0.9909 - loss: 0.4449
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - binary_accuracy: 0.9923 - loss: 0.4046
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - binary_accuracy: 0.9969 - loss: 0.3725
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - binary_accuracy: 1.0000 - loss: 0.3498
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - binary_accuracy: 0.9999 - loss: 0.3282
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - binary_accuracy: 1.0000 - loss: 0.3162
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - binary_accuracy: 1.0000 - loss: 0.3029
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - binary_accuracy: 1.0000 - loss: 0.2966
